In [ ]:
%matplotlib widget

In [ ]:
from astropy.io import fits
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import IntSlider, interact, VBox
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm

## Loading the data

(requirement #1)

In [ ]:
name = '/eng/ssb/busko/NGC5406.V1200.rscube.fits'
hdulist = fits.open(name)
if len(hdulist)>1:
    hdu=hdulist[1]
else:
    hdu=hdulist[0]

cube = hdu.data.T # remember to always transpose with fits files
hdr = hdu.header
hdr['CRVAL3'] = 0 # adapt to this data
hdr['Cd3_3'] = 1

# Data seems to have lots of garbage at the edges. It has also
# a lot of bad pixels. These should be handled by a masked array
# version of the cube, but for now we resort to clipping and
# forcing the plot axis and grayscales to behave.  

In [ ]:
print(cube.shape)

## Calculate the median map

In [ ]:
median = np.median(cube, axis=2)

## Display cube, median map, and spectra

(requirements #2 and #3)

In [ ]:
# Create figure and axes
fig_cube = plt.figure(figsize=(8,5))
ax_cube = fig_cube.add_subplot(221)
ax_median = fig_cube.add_subplot(222)
ax_plot = fig_cube.add_subplot(223)

ax_cube.set_xticks([])
ax_cube.set_yticks([])
ax_median.set_xticks([])
ax_median.set_yticks([])
ax_cube.set_title('3D Data Cube')
ax_median.set_title('Median Map')

# Display the raw cube and median maps
median_im = ax_median.imshow(median, vmin=0.,vmax=0.02)
divider = make_axes_locatable(ax_median)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig_cube.colorbar(median_im, cax=cax)

cube_im = ax_cube.imshow(cube[:,:,0],vmin=0.,vmax=0.02,cmap=cm.get_cmap('Greys'))
divider = make_axes_locatable(ax_cube)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig_cube.colorbar(cube_im, cax=cax)

plt.tight_layout()

# Create the slider and its callback
slider = IntSlider(value=int(cube.shape[2]/2.), min=0, max=cube.shape[2]-1)

def plot_test(slice_num):
    cube_im.set_data(cube[:,:,slice_num])
    fig_cube.canvas.draw_idle()

_ = interact(plot_test, slice_num=slider)

# Plot spectrum in the center of the cube, and add
# the 1D plotting interactivity to the cube image.
ax_plot.set_ylim([0.0,0.1])
ax_plot.plot(cube[int(cube.shape[0]/2),int(cube.shape[1]/2),:])

def onclick(event):
    if event.inaxes == ax_cube:
        ax_plot.clear()
        ax_plot.set_ylim([0.0,0.1])
        ax_plot.plot(cube[int(event.xdata),int(event.ydata),:])
        
cid = fig_cube.canvas.mpl_connect('button_press_event', onclick)

VBox([fig_cube.canvas, slider])

End of demo.